In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
!git clone https://github.com/rodrigorivera/mds20_deepfolio

from mds20_deepfolio.models.NeuralHawkesProcess.DataWrapper import NHPDataset, collate_fn
from mds20_deepfolio.models.NeuralHawkesProcess.model import NHPModel
from mds20_deepfolio.models.NeuralHawkesProcess.train import train

!unzip /content/mds20_deepfolio/models/NeuralHawkesProcess/data/fin_data.zip \
      -d /content/mds20_deepfolio/models/NeuralHawkesProcess/data/


Cloning into 'mds20_deepfolio'...
remote: Enumerating objects: 165, done.
remote: Counting objects: 100% (165/165), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 616 (delta 102), reused 0 (delta 0), pack-reused 451
Receiving objects: 100% (616/616), 19.58 MiB | 10.11 MiB/s, done.
Resolving deltas: 100% (337/337), done.
Archive:  /content/mds20_deepfolio/models/NeuralHawkesProcess/data/fin_data.zip
  inflating: /content/mds20_deepfolio/models/NeuralHawkesProcess/data/test.pkl  
  inflating: /content/mds20_deepfolio/models/NeuralHawkesProcess/data/dev.pkl  
  inflating: /content/mds20_deepfolio/models/NeuralHawkesProcess/data/train.pkl  


In [3]:
train_dataset = NHPDataset('/content/mds20_deepfolio/models/NeuralHawkesProcess/data/train.pkl')
val_dataset = NHPDataset('/content/mds20_deepfolio/models/NeuralHawkesProcess/data/dev.pkl')
test_dataset = NHPDataset('/content/mds20_deepfolio/models/NeuralHawkesProcess/data/test.pkl')

train_loader = DataLoader(train_dataset, batch_size=12, collate_fn=collate_fn)
val_loader = DataLoader(val_dataset, batch_size=12, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=12, collate_fn=collate_fn)

print('lenght of train_dataset:', len(train_dataset))
print('lenght of val_dataset:', len(val_dataset))
print('lenght of test_dataset:', len(test_dataset))

lenght of train_dataset: 90
lenght of val_dataset: 10
lenght of test_dataset: 100


In [4]:
model = NHPModel(256, device=device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

statiscs = train(model, optimizer, train_loader, val_loader, device, n_epochs = 50, sum_losses=True)

Epoch: 0
Log-Likelihood:: train: -2.6600452780288575 , val: -1.3004566510997289
Time MSE:: train: 386.0933728814125 , val: 1.8944569826126099
Event CE:: train: 0.7028402760624886 , val: 0.6913579702377319
Event pred accuracy:: train: 0.4820894866385373 , val: 0.5378842676311031
time: 61.0845992565155
------------------------------------------------------------
Epoch: 1
Log-Likelihood:: train: -2.5492988666369554 , val: -1.1612732797905996
Time MSE:: train: 385.9461745470762 , val: 1.8504773378372192
Event CE:: train: 0.6873300895094872 , val: 0.6804768443107605
Event pred accuracy:: train: 0.5517442736588306 , val: 0.5971971066907775
time: 122.02376890182495
------------------------------------------------------------
Epoch: 2
Log-Likelihood:: train: -2.5442181719937396 , val: -1.1467964983805363
Time MSE:: train: 385.81519958376884 , val: 1.8075306415557861
Event CE:: train: 0.6748963817954063 , val: 0.6722013354301453
Event pred accuracy:: train: 0.6147560026120152 , val: 0.598673899

KeyboardInterrupt: ignored

In [22]:
loss_time, type_acc = 0, 0
for event_seq, time_seq in test_loader:
    event_seq, time_seq = event_seq.to(device), time_seq.to(device)
    intens, time, event = model.forward(event_seq, time_seq)
    loss_time += model.time_loss(time, time_seq)
    type_acc += accuracy_score(event[:,:-1].argmax(dim=2).cpu().reshape(-1), 
                                                  event_seq[:, 1:].cpu().reshape(-1))



In [35]:
print('MSE error on test dataset', (loss_time/len(test_loader)).item())
print('Type accuract on test dataset', type_acc/len(test_loader))

MSE error on test dataset 281.4898681640625
Type accuract on test dataset 0.6164117015566292


In [28]:
from sklearn.metrics import accuracy_score, mean_squared_error

def evaluate_prediction(model, dataloader, device):
        """
        Evalute prediction on give dataset
        Will compute mse and accuracy score for event time and type prediction.
        Input:
           model - NHP model to compute decay states for sequence
           dataloader - dataloader with data
        Output:
           mean_squared_error - for event time prediction
           accuracy_score - for event type prediction
        """
        pred_data = []
        for event_seqs, time_seqs in dataloader:
            for i in range(len(event_seqs)):
                pred_data.append(predict_event(model, time_seqs[i], event_seqs[i], len(time_seqs[i])-1, device)[:4])

        pred_data = np.array(pred_data)
        time_gt, time_pred = pred_data[:,0], pred_data[:,1]
        type_gt, type_pred = pred_data[:,2], pred_data[:,3]

        time_mse_error = mean_squared_error(time_gt, time_pred)
        type_accuracy = accuracy_score(type_gt, type_pred)

        return time_mse_error, type_accuracy

def predict_event(model, seq_time, seq_events, seq_length, device, hmax = 40,
                     n_samples=1000):
        """ 
        Predict last event time and type for the given sequence 
        Last event takes as unknown and model feeds with all remain sequence.
        Input:
            model - NHP model to compute decay states for sequence
            seq_time - torch.tensor with time diffs between events in sequence
            seq_events - torch.tensor with event types for each time point
            seq_length - length of the sequence
        
        Output:
            pred_dt - predicted dt for next event
            gt_dt - gt df of next event
            pred_type - predicted type of next event
            gt_type - gt_type of next event
            time_between_events - np.array - generated timestamps
            intensity - np.array - intensity after event
        """

        """ Feed the model with sequence and compute decay cell state """

        with torch.no_grad():
            model.init_states(1)
            for i in range(seq_length):
                c_t, c_target, output, decay = model.CTLSTM_cell(model.Embedding(seq_events[i].to(device)).unsqueeze(0), model.hidden_decay, 
                                                               model.cell_decay, model.cell_target)

                if i < seq_length - 1:

                    c_t = c_t * torch.exp(-decay * seq_time[i, None].to(device)) 
                    h_t = output * torch.tanh(c_t)

            # gt last and one before last event types and times
            last_type, gt_type = seq_events[i], seq_events[i + 1]
            gt_dt = seq_time[i]


            """ Make prediction for the next event time and type """
            model.eval()
            timestep = hmax / n_samples

            # 1) Compute intensity
            time_between_events = torch.linspace(0, hmax, n_samples + 1).to(device)
            hidden_vals = h_t * torch.exp(-decay * time_between_events[:, None])
            intensity = model.intensity_layer(hidden_vals.to(device))
            intensity_sum = intensity.sum(dim=1)


            # 2) Compute density via integral 
            density = torch.cumsum(timestep * intensity.sum(dim=1), dim=0)
            density = intensity_sum * torch.exp(-density)

            # 3) Predict time of the next event via trapeze method
            t = time_between_events * density   
            pred_dt = (timestep * 0.5 * (t[1:] + t[:-1])).sum() 
            # 4) Predict type of the event via trapeze method
            P = intensity / intensity_sum[:, None] * density[:, None]  
            pred_type = torch.argmax(timestep * 0.5 * (P[1:] + P[:-1])).sum(dim=0)

            return pred_dt.cpu().numpy(), gt_dt.cpu().numpy(), pred_type.cpu().numpy(), gt_type.cpu().numpy(), \
                            time_between_events.cpu().numpy(), intensity.cpu().numpy()


In [29]:
time_mse_error_test, type_accuracy_test = evaluate_prediction(model, test_loader,device)


In [30]:
time_mse_error_test, type_accuracy_test

(20.796348920741174, 0.46)